In [1]:
import os
import argparse
import torch
import random
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
import torch.nn.functional as F
from torch.utils.data import random_split
import clip
import loraclip
from torchvision import datasets, transforms
from transformers import ViTImageProcessor, ViTForImageClassification
from datasets import load_dataset
from encoder_utils import build_faiss_index, predict_with_faiss, compute_topk_accuracy, CLIPClassifier
from tqdm import tqdm
from dotenv import load_dotenv
from gym import Trainer
import wandb

/opt/conda/envs/python3.10-la-proj/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: Currently logged in as: andriy-suh (andriy-suh-private) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_vit16_model, clip_preprocess = clip.load("ViT-B/16", device=device)

In [3]:
path_data = "./data"
os.makedirs(path_data, exist_ok=True)

In [4]:
cifar100_dataset_train = datasets.CIFAR100(root=path_data, train=True, download=True, transform=clip_preprocess)
cifar100_dataset_test = datasets.CIFAR100(root=path_data, train=False, download=True, transform=clip_preprocess)

train_size = int(len(cifar100_dataset_train) * 0.8)
val_size = len(cifar100_dataset_train) - train_size
cifar100_train_x, cifar100_val_x = random_split(cifar100_dataset_train, [train_size, val_size])

cifar100_loader_train = DataLoader(cifar100_train_x, batch_size=128, shuffle=True)
cifar100_loader_val = DataLoader(cifar100_val_x, batch_size=128, shuffle=False)
cifar100_loader_test = DataLoader(cifar100_dataset_test, batch_size=128, shuffle=False)

## ViT B/16: Zero-shot prediction

In [7]:
faiss_labels, faiss_index = build_faiss_index(
    dataloader=cifar100_loader_train,
    model=clip_vit16_model,
    device=device
)

Building FAISS Index: 100%|██████████| 625/625 [15:01<00:00,  1.44s/it]


FAISS index built with 40000 entries.


In [8]:
ground_truth, predictions = predict_with_faiss(
    dataloader=cifar100_loader_test,
    model=clip_vit16_model,
    faiss_index=faiss_index,
    faiss_labels=faiss_labels,
    device=device,
    top_k=5,
    distractor_classes=None
)

Predicting with FAISS: 100%|██████████| 157/157 [06:10<00:00,  2.36s/it]


In [9]:
accuracy_top1 = compute_topk_accuracy(ground_truth, predictions, top_k=1)
print('top 1 accuracy', accuracy_top1)

top 1 accuracy 0.6912


In [10]:
accuracy_top2 = compute_topk_accuracy(ground_truth, predictions, top_k=2)
print('top 2 accuracy', accuracy_top2)

top 2 accuracy 0.8046


## ViT B/16: Train LoRA Adapter R=4

In [ ]:
clip_vit16_lora_r4 = 
clip_vit16_cl_model = CLIPClassifier(clip_vit16_model, fine_tune=True).float().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(clip_vit16_cl_model.parameters(), lr=1e-4)

trainer = Trainer(
    model=clip_vit16_cl_model,
    train_loader=cifar100_loader_train,
    val_loader=cifar100_loader_val,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    num_epochs=10,
    hf_repo_name="ansu0122/vit-lora",
    experiment_name="clip_vit16_lora_r4",
    project_name="lora-project"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [ ]:
trainer.train()

In [17]:
wandb.finish()

## ViT B/16: Train LoRA Adapter R=16

## Release Memory

In [ ]:
import torch
del clip_vit16_cl_model
del trainer
torch.cuda.empty_cache()

In [40]:
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

In [41]:
for var in dir():
    if isinstance(eval(var), torch.Tensor):
        del globals()[var]
torch.cuda.empty_cache()

In [16]:
!nvidia-smi

Sun Feb 16 23:24:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.05             Driver Version: 550.127.05     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090        On  |   00000000:62:00.0 Off |                  N/A |
| 30%   44C    P8             24W /  350W |    1548MiB /  24576MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [45]:
torch.cuda.empty_cache()

gc.collect()

with torch.no_grad():
    torch.cuda.empty_cache()

print(f"Memory allocated: {torch.cuda.memory_allocated() / 1024 / 1024:.2f} MB")
print(f"Memory reserved: {torch.cuda.memory_reserved() / 1024 / 1024:.2f} MB")

Memory allocated: 584.70 MB
Memory reserved: 634.00 MB
